# A/B Testing Framework - Демонстрация

Этот notebook демонстрирует возможности фреймворка для A/B тестирования.

## Содержание:
1. [Импорт библиотек](#imports)
2. [Тест конверсии](#conversion-test)
3. [Тест непрерывной метрики](#continuous-test)
4. [Расчёт размера выборки](#sample-size)
5. [Анализ мощности теста](#power-analysis)

## 1. Импорт библиотек {#imports}

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

# Импортируем наш модуль
from ab_testing import ABTesting, TestType
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Настройка отображения
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = (12, 8)

print("Все библиотеки успешно импортированы!")

## 2. Тест конверсии {#conversion-test}

Сравниваем конверсию двух версий лендинга:

In [ ]:
# Инициализация A/B теста
ab_test = ABTesting(alpha=0.05)

# Генерируем данные для теста конверсии
conversion_data = ab_test.generate_sample_data(
    n_control=1500,    # Старая версия сайта
    n_treatment=1500,  # Новая версия сайта
    test_type=TestType.CONVERSION,
    effect_size=0.025  # Ожидаемое увеличение конверсии на 2.5%
)

print(f"Данные сгенерированы:")
print(f"• Общий размер выборки: {len(conversion_data)}")
print(conversion_data.head())

In [ ]:
# Базовый анализ данных
summary = conversion_data.groupby('group')['converted'].agg(['count', 'sum', 'mean'])
summary.columns = ['Пользователей', 'Конверсий', 'Конверсия %']
summary['Конверсия %'] = summary['Конверсия %'].round(4)

print("Предварительная статистика:")
print(summary)

# Рассчитаем относительное изменение
conv_a = summary.loc['A', 'Конверсия %']
conv_b = summary.loc['B', 'Конверсия %']
relative_change = ((conv_b / conv_a - 1) * 100)

print(f"\nОтносительное изменение: {relative_change:+.2f}%")

In [ ]:
# Проводим статистический тест
conversion_result = ab_test.conversion_test(conversion_data, 'converted')

# Выводим подробный отчёт
print(ab_test.generate_report(conversion_result))

In [ ]:
# Визуализируем результаты
ab_test.plot_results(conversion_data, conversion_result)
plt.show()

## 3. Тест непрерывной метрики {#continuous-test}

Сравниваем среднюю выручку на пользователя:

In [ ]:
# Генерируем данные для теста выручки
revenue_data = ab_test.generate_sample_data(
    n_control=1000,
    n_treatment=1000,
    test_type=TestType.CONTINUOUS,
    effect_size=0.12  # Увеличение выручки на 12%
)

print("Данные по выручке:")
revenue_summary = revenue_data.groupby('group')['revenue'].agg(['count', 'mean', 'std'])
revenue_summary.columns = ['Пользователей', 'Средняя выручка', 'Стд. отклонение']
revenue_summary = revenue_summary.round(2)

print(revenue_summary)

# Относительное изменение
rev_change = ((revenue_summary.loc['B', 'Средняя выручка'] / revenue_summary.loc['A', 'Средняя выручка'] - 1) * 100)
print(f"\nИзменение выручки: {rev_change:+.2f}%")

In [ ]:
# Проводим t-тест
revenue_result = ab_test.continuous_test(revenue_data, 'revenue')

print(ab_test.generate_report(revenue_result))

In [ ]:
# Визуализация результатов
ab_test.plot_results(revenue_data, revenue_result, metric_col='revenue')
plt.show()

## 4. Расчёт размера выборки {#sample-size}

Определяем необходимый размер выборки перед началом эксперимента:

In [ ]:
# Различные сценарии для планирования экспериментов
scenarios = [
    {"name": "Высокая конверсия, малый эффект", "baseline": 0.15, "lift": 0.05},
    {"name": "Средняя конверсия, средний эффект", "baseline": 0.08, "lift": 0.15},
    {"name": "Низкая конверсия, большой эффект", "baseline": 0.02, "lift": 0.30},
]

print("📏 ПЛАНИРОВАНИЕ РАЗМЕРА ВЫБОРКИ")
print("=" * 50)

sample_sizes = []

for scenario in scenarios:
    sample_size = ab_test.sample_size_calculator(
        baseline_rate=scenario["baseline"],
        expected_lift=scenario["lift"],
        power=0.8,
        alpha=0.05
    )
    
    sample_sizes.append({
        'Сценарий': scenario['name'],
        'Базовая конверсия': f"{scenario['baseline']:.1%}",
        'Ожидаемый прирост': f"{scenario['lift']:.1%}",
        'Размер группы': f"{sample_size:,}",
        'Общий размер': f"{sample_size * 2:,}"
    })

sample_df = pd.DataFrame(sample_sizes)
print(sample_df.to_string(index=False))

In [ ]:
# Визуализация зависимости размера выборки от параметров
baselines = np.arange(0.01, 0.21, 0.01)
lifts = [0.05, 0.10, 0.15, 0.20, 0.25]

fig, ax = plt.subplots(figsize=(12, 8))

for lift in lifts:
    sizes = []
    for baseline in baselines:
        try:
            size = ab_test.sample_size_calculator(baseline, lift, power=0.8)
            sizes.append(size)
        except:
            sizes.append(np.nan)
    
    ax.plot(baselines, sizes, label=f'Прирост {lift:.0%}', linewidth=2)

ax.set_xlabel('Базовая конверсия')
ax.set_ylabel('Размер выборки (каждая группа)')
ax.set_title('Зависимость размера выборки от базовой конверсии и эффекта')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 50000)

plt.tight_layout()
plt.show()

## 5. Анализ мощности теста {#power-analysis}

Исследуем как размер выборки влияет на способность обнаруживать эффекты:

In [ ]:
# Симуляция экспериментов с различными размерами выборки
sample_sizes = [100, 300, 500, 1000, 2000, 3000]
effect_size = 0.03  # Фиксированный эффект 3%

results = []

for n in sample_sizes:
    # Генерируем данные с фиксированным эффектом
    data = ab_test.generate_sample_data(
        n_control=n,
        n_treatment=n,
        test_type=TestType.CONVERSION,
        effect_size=effect_size
    )
    
    result = ab_test.conversion_test(data)
    
    results.append({
        'Размер выборки': n,
        'Общий размер': n * 2,
        'p-value': result.p_value,
        'Мощность': result.power,
        'Значимо': result.is_significant,
        'Конверсия A': result.group_a_metric,
        'Конверсия B': result.group_b_metric
    })

power_df = pd.DataFrame(results)
print("АНАЛИЗ МОЩНОСТИ ДЛЯ РАЗЛИЧНЫХ РАЗМЕРОВ ВЫБОРКИ")
print("=" * 60)
print(power_df.round(4))

In [ ]:
# Визуализация мощности
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# График мощности
ax1.plot(power_df['Размер выборки'], power_df['Мощность'], 'bo-', linewidth=2, markersize=8)
ax1.axhline(y=0.8, color='red', linestyle='--', label='Рекомендуемая мощность (0.8)')
ax1.set_xlabel('Размер выборки (каждая группа)')
ax1.set_ylabel('Мощность теста')
ax1.set_title('Зависимость мощности от размера выборки')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_ylim(0, 1)

# График p-values
colors = ['red' if p < 0.05 else 'gray' for p in power_df['p-value']]
ax2.scatter(power_df['Размер выборки'], power_df['p-value'], c=colors, s=100, alpha=0.7)
ax2.axhline(y=0.05, color='red', linestyle='--', label='α = 0.05')
ax2.set_xlabel('Размер выборки (каждая группа)')
ax2.set_ylabel('p-value')
ax2.set_title('p-values для различных размеров выборки')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_yscale('log')

plt.tight_layout()
plt.show()

## Заключение

В этом notebook мы продемонстрировали:

1. **Тесты конверсии** - для биномиальных метрик (конверсия, CTR)
2. **Тесты непрерывных метрик** - для средних значений (выручка, время на сайте)
3. **Планирование экспериментов** - расчёт необходимого размера выборки
4. **Анализ мощности** - понимание вероятности обнаружения реальных эффектов

### Рекомендации:

- **Всегда планируйте размер выборки** заранее на основе ожидаемого эффекта
- **Стремитесь к мощности 0.8+** для надёжного обнаружения эффектов
- **Используйте доверительные интервалы** для оценки практической значимости
- **Анализируйте сегменты** для более глубокого понимания результатов